In [ ]:
# Various torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

# torchvision
from torchvision import datasets, transforms

# ------------------------
# get up one directory 
import sys, os
sys.path.append(os.path.abspath('../'))
# ------------------------

# custom packages
import models.aux_funs as maf
import optimizers as op
import regularizers as reg
import train
import math
import utils.configuration as cf
import utils.datasets as ud
from models.fully_connected import fully_connected

# Fix the random seed

In [ ]:
random_seed = 2
cf.seed_torch(random_seed)

# Configure the experiment

In [ ]:
sparse_init = 0.01
r = [1,0.7/math.sqrt(sparse_init)]

conf_args = {#
    # data specification
    'data_file':"../../Data", 'train_split':0.80, 'data_set':"MNIST", 'download':True,
    # cuda
    'use_cuda':False, 'num_workers':0, 'cuda_device':0, 'pin_memory':True, 'train_split':0.80,
    #
    'epochs':7,
    # optimizer
    'delta':1.0, 'lr':0.1, 'lamda':1e-1, 'optim':"LinBreg",'beta':0.0,
    # initialization
    'sparse_init':sparse_init, 'r':r,
    # misc
    'random_seed':random_seed, 'eval_acc':True,
}

conf = cf.Conf(**conf_args)

# Initiate the model

In [ ]:
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std}    

# Determine input size based on dataset
if conf.data_set == "CIFAR10":
    input_size = 3072  # 32x32x3
elif conf.data_set in ["MNIST", "Fashion-MNIST"]:
    input_size = 784   # 28x28x1
else:
    input_size = 784  # Default fallback
    
sizes = [input_size, 200, 80, 10]  # Dynamically set input size
act_fun = torch.nn.ReLU()
    
model = fully_connected(sizes, act_fun, **model_kwargs)
best_model = train.best_model(fully_connected(sizes, act_fun, **model_kwargs).to(conf.device))

# Weight initialization

In [ ]:
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std}    
def init_weights(conf, model):
    # sparsify
    maf.sparse_bias_uniform_(model, 0,conf.r[0])
    maf.sparse_weight_normal_(model, conf.r[1])
    
    maf.sparsify_(model, conf.sparse_init)
    model = model.to(conf.device)
    return model

model = init_weights(conf,model)

# Optimizer

In [ ]:
def init_opt(conf, model):
    weights_linear = maf.get_weights_linear(model)
    biases = maf.get_bias(model)

    if conf.optim == "SGD":
        opt = torch.optim.SGD(model.parameters(), lr=conf.lr, momentum=conf.beta)
    elif conf.optim == "LinBreg":
        opt = op.LinBreg([{'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda), 'momentum':conf.beta, 'delta':conf.delta},
                          {'params': biases, 'lr': conf.lr, 'momentum':conf.beta}])
    elif conf.optim == "LinBregHeavyBall":
        opt = op.LinBregHeavyBall([{'params': weights_linear, 'lr': conf.lr,
                                    'reg': reg.reg_l1(lamda=conf.lamda), 'momentum': conf.beta,
                                    'delta': conf.delta},
                                   {'params': biases, 'lr': conf.lr, 'momentum': conf.beta}])
    elif conf.optim == "LinBregNesterov":
        opt = op.LinBregNesterov([{'params': weights_linear, 'lr': conf.lr,
                                  'reg': reg.reg_l1(lamda=conf.lamda), 'momentum': conf.beta,
                                  'delta': conf.delta},
                                 {'params': biases, 'lr': conf.lr, 'momentum': conf.beta}])
    elif conf.optim == "adam":
        opt = op.AdaBreg([{'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda)},
                          {'params': biases, 'lr': conf.lr}])
    elif conf.optim == "ProxSGD":
        opt = op.ProxSGD([{'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda)},
                          {'params': biases, 'lr': conf.lr}])
    else:
        raise ValueError("Unknown Optimizer specified")

    # learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=5,threshold=0.01)
    
    return opt, scheduler

# Dataset

In [ ]:
train_loader, valid_loader, test_loader = ud.get_data_set(conf)

# History and Runs

In [ ]:
# Initialize history
tracked = ['acc', 'loss', 'linear_sparse', 'reg_vals']

def reset_hist(tracked):
    train_hist = {}
    val_hist = {}
    return train_hist, val_hist

params = [
    # SGD Run (LinBreg with lamda=0.0, beta=0.0)
    {'optim': 'LinBreg', 'reps': 1, 'lamda': 0.0, 'random_seed': 0, 'label': 'SGD'},

    # LinBreg Run (beta=0.0)
    {'optim': 'LinBreg', 'reps': 1, 'lamda': 1e-1, 'beta': 0.0, 'random_seed': 0, 'label': 'LinBreg ($\lambda=1$e-1)'},

    # LinBregNesterov Run (beta=0.9)
    {'optim': 'LinBregNesterov', 'reps': 1, 'lamda': 1e-1, 'beta': 0.5, 'random_seed': 0, 'label': 'LinBregNesterov ($\lambda=1$e-3, $\\beta=0.5$)'},

    # AdaBreg Run
    {'optim': 'adam', 'reps': 1, 'lamda': 1e-1, 'random_seed': 0, 'label': 'AdaBreg ($\\lambda=1$e-1)'},

 ]
runs = cf.run(params)

# Training

In [ ]:
while runs.step(conf):
    # -----------------------------------------------------------------------------------
    # Reinit weights and the corresponding optimizer
    # -----------------------------------------------------------------------------------
    train_hist, val_hist = reset_hist(tracked)
    model = init_weights(conf, model)
    opt, scheduler = init_opt(conf, model)
    
    # -----------------------------------------------------------------------------------
    # train the model
    # -----------------------------------------------------------------------------------
    for epoch in range(conf.epochs):
        print(25*"<>")
        print(50*"|")
        print(25*"<>")
        print('Epoch:', epoch)

        # ------------------------------------------------------------------------
        # train step, log the accuracy and loss
        # ------------------------------------------------------------------------
        train_data = train.train_step(conf, model, opt, train_loader)

        # update history
        for key in tracked:
            if key in train_data:
                var_list = train_hist.setdefault(key, [])
                var_list.append(train_data[key])        

        # ------------------------------------------------------------------------
        # validation step
        val_data = train.validation_step(conf, model, opt, valid_loader)

        # update history
        for key in tracked:
            
            
            if key in val_data:
                var = val_data[key]
                if isinstance(var, list):
                    for i, var_loc in enumerate(var):
                        key_loc = key+"_" + str(i)
                        var_list = val_hist.setdefault(key_loc, [])
                        val_hist[key_loc].append(var_loc)
                else:
                    var_list = val_hist.setdefault(key, [])
                    var_list.append(var)    

        # scheduler step
        scheduler.step(train_data['loss'])
        print("Learning rate:",opt.param_groups[0]['lr'])
        
        # update beset model
        best_model(train_data['acc'], val_data['acc'], model=model)

        
    # add values to the run history
    runs.add_history(train_hist, "train")
    runs.add_history(val_hist, "val")
            
    # update random seed
    cf.seed_torch(conf.random_seed)

In [ ]:
if runs.history:
    print("Keys available in the first run's history:")
    print(runs.history[0].keys())
else:
    print("No history recorded yet.")

# Prepare Data
In this step we average over different runs of the same parameter configuration.

In [ ]:
import numpy as np

hist = runs.history
keys = ['val_acc', 'val_linear_sparse']
hist_idx = 0
for param in params:
    data = {}
    for key in keys:
        if not key in hist[hist_idx]:
            continue
        
        if key == 'train_acc' or key == 'val_acc' or key == 'val_linear_sparse':
            rescale = 100
        else:
            rescale = 1/param['lamda'] if param['lamda'] > 0.0 else 0.0
            
        n = len(hist[hist_idx][key])
        m = param.get('reps',1)
        data_loc = np.zeros(shape=(n,m))
        
        # assign data and save it into local array for mean and average
        for i in range(m):
            var = np.array(hist[hist_idx + i][key])
            data_loc[:,i] = rescale*var
            data[key+"_run_" + str(i)] = rescale*var

        # mean and std of the data
        data[key+"_mean"] = np.mean(data_loc,axis=1)
        data[key+"_std"] = np.std(data_loc,axis=1)

                # --- Add this print statement ---
        if key == 'train_acc':
            print(f"  Debug: Successfully created 'train_acc_mean'. Length: {len(data[key+'_mean'])}")
        # --- End of added print statement ---
        
        param['result'] = data
        
        # update the history index
    hist_idx += m

# Setup plots and appearance

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('default')
plt.style.use('ggplot')
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size']=8
matplotlib.rcParams['lines.linewidth'] = 1
matplotlib.rcParams['lines.markersize'] = 2
matplotlib.rcParams['text.color'] = 'black'

In [ ]:

def plot_training_stats(ax, keys_to_plot, data_dict, label='', color='k',alpha=1.0, alpha_fill=0.2):
    # keys_to_plot should be ['train_acc', 'val_acc', 'val_reg_vals_0', 'val_linear_sparse']
    # data_dict is param['result']
    
    for i in range(len(keys_to_plot)):
        current_key = keys_to_plot[i]
        mean_key = current_key + '_mean'
        std_key = current_key + '_std'
        
        # Debug print specifically for the first axis (train_acc)
        if i == 0:
            print(f"  Plotting Function Check (ax[0], label='{label}'): Trying to plot key '{current_key}'. Looking for '{mean_key}' in data_dict.")
            if mean_key in data_dict:
                 print(f"    '{mean_key}' found. Data length: {len(data_dict[mean_key])}")
            else:
                 print(f"    '{mean_key}' NOT FOUND in data_dict!")


        if not mean_key in data_dict:
            # print(f"    Skipping {mean_key} for label '{label}' - not found in data.") # Optional: uncomment for more verbose output
            continue
        if not std_key in data_dict:
            # print(f"    Skipping {std_key} for label '{label}' - not found in data.") # Optional: uncomment for more verbose output
            continue
            
        # --------------------------------
        var_mean = data_dict[mean_key]
        var_std = data_dict[std_key]
        # --------------------------------
        
        # Ensure data is not empty
        if len(var_mean) == 0:
            print(f"    Warning: '{mean_key}' for label '{label}' is empty. Skipping plot.")
            continue

        epochs = np.arange(len(var_mean))
        
        # Check target axis index is valid
        if i < len(ax):
             target_ax = ax[i]
             target_ax.plot(epochs,var_mean, label=label, color=color,alpha=alpha)
             target_ax.fill_between(epochs, var_mean - var_std, var_mean + var_std, color=color, alpha=alpha_fill)
        else:
            print(f"    Warning: Axis index {i} out of bounds for label '{label}'.")


# Colors

In [ ]:
cmp = matplotlib.cm.get_cmap(name='Accent') 

colors = [
    cmp(0.0), # SGD (e.g., blue)
    cmp(0.7), # LinBreg (e.g., green)
    'red',    # LinBregNesterov (red)
    cmp(0.2), # AdaBreg (e.g., orange)
]

if len(colors) < len(params):
    num_missing = len(params) - len(colors)
    for i in range(num_missing):
        colors.append(cmp( (len(colors) + i) * 0.1 % 1.0 )) 

for i, param in enumerate(params):
    param['color'] = colors[i]
    param.setdefault('label', param['optim'])

# Final Plot

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))  
ax = np.ravel(ax)  

plt.rcParams['font.size'] = 26            
plt.rcParams['axes.labelsize'] = 36       
plt.rcParams['xtick.labelsize'] = 36      
plt.rcParams['ytick.labelsize'] = 36      
plt.rcParams['legend.fontsize'] = 36      
plt.rcParams['lines.linewidth'] = 3.5     

params[0]['label'] = 'SGM'  
params[1]['label'] = 'LinBreg ($\lambda=1$e-1)'
params[2]['label'] = 'LinBregNesterov ($\lambda=5$e-2, $\\beta=0.5$)'

for param in params:
    plot_training_stats(ax, keys, param['result'], color=param['color'], label=param['label'])
    
ax[0].set_ylabel('Validation Accuracy [%]', fontsize=40)  
ax[0].set_xlabel('Epoch', fontsize=40)  

ax[1].set_ylabel('Non-Zero Entries [%]', fontsize=40)  
ax[1].set_xlabel('Epoch', fontsize=40)  

ax[0].set_ylim(bottom=95.0, top=98.5)
yticks = np.arange(95.0, 99.0, 0.5)
ax[0].set_yticks(yticks)
ax[0].set_yticklabels([f'{y:.1f}' for y in yticks], fontsize=30)  

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), 
           prop={'size': 32}, ncol=len(params))  

plt.tight_layout()
plt.subplots_adjust(top=0.82, wspace=0.3)  


In [ ]:
import pandas as pd

optimizer_labels = [
    'SGD',
    'LinBreg ($\lambda=1$e-1)',
    'LinBregNesterov ($\lambda=1$e-3, $\\beta=0.5$)',
    'AdaBreg ($\lambda=1$e-1)',
]

records = []

# Each run's history is stored in runs.history.
# We assume that for each run:
#    - run['label'] is the optimizer label (e.g., "LinBreg ($\lambda=1$e-3)")
#    - run['lamda'] is the regularization parameter value
#    - run['train_acc'] and run['val_acc'] store the final accuracies.
#    - run['val_linear_sparse'] stores the sparsity level.
for i, run in enumerate(runs.history):
    label = optimizer_labels[i % len(optimizer_labels)]  # Use modulo to handle repetitions
    sparsity = run.get('val_linear_sparse', [None])[-1]
    train_acc = run.get('train_acc', [None])[-1]
    val_acc = run.get('val_acc', [None])[-1]
    
    records.append({
        'Optimizer': label,
        'Sparsity Level': sparsity,
        'Train Accuracy': train_acc,
        'Validation Accuracy': val_acc
    })

df = pd.DataFrame(records)
print(df)